In [1]:
%reset -f

In [2]:
import pandas as pd
from datetime import datetime
from querys import get_cuenta_npl
from load import export_dataframe_to_sql
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
current_month_year = datetime.now().strftime("%m.%Y")
source = "payments.xlsx"

In [4]:
df = pd.read_excel(source, sheet_name=current_month_year, header=0, usecols="A,D,E,G,H,J")
df = df.dropna(how='all', axis=1).dropna(how='all').reset_index(drop=True)
df.columns = ['FECHA', 'PAGO', 'BANCO', 'CONTRATO', 'DNI', 'CARTERA']
df = df[df['DNI'] != 0]
df['CONTRATO'] = df['CONTRATO'].apply(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else x)
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y', errors='coerce')
df['PAGO'] = df['PAGO'].round(2)

In [5]:
df['CONTRATO'] = df['CONTRATO'].str.strip()  # Quitar espacios al inicio y al final
df['CONTRATO'] = df['CONTRATO'].apply(lambda x: x.zfill(10))  # Agregar ceros a la izquierda hasta 10 caracteres

In [6]:
df['CUENTA'] = df['CONTRATO'].apply(lambda contrato: get_cuenta_npl(contrato))

In [9]:
print(df.dtypes)

FECHA       datetime64[ns]
PAGO               float64
BANCO               object
CONTRATO            object
DNI                  int64
CARTERA             object
CUENTA              object
dtype: object


In [10]:
export_dataframe_to_sql(df, "NPL_PAYMENTS", schema='cargue')

Error al exportar los datos: (pyodbc.ProgrammingError) ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The specified schema name "cargue" either does not exist or you do not have permission to use it. (2760) (SQLExecDirectW)')
[SQL: 
CREATE TABLE cargue.[NPL_PAYMENTSs] (
	[FECHA] DATETIME NULL, 
	[PAGO] FLOAT(53) NULL, 
	[BANCO] VARCHAR(max) NULL, 
	[CONTRATO] VARCHAR(max) NULL, 
	[DNI] BIGINT NULL, 
	[CARTERA] VARCHAR(max) NULL, 
	[CUENTA] VARCHAR(max) NULL
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [16]:
print(df.dtypes)

FECHA       datetime64[ns]
PAGO               float64
BANCO               object
CONTRATO            object
DNI                 object
CARTERA             object
CUENTA              object
dtype: object
